In [9]:
import torch
import numpy as np
import random
import networkx as nx
import os

def set_seed(seed):

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def get_random_problems(batch_size, problem_size, original_node_count, link_count, seed=1234, save_path=None):

    set_seed(seed)
    
    train_s, train_d, adj_matrices = generate_network_batch(batch_size, problem_size,
                                                            original_node_count, link_count, seed=seed)
    depot_xy = train_s[:, 0:1, :] 
    node_xy = train_s[:, 1:, :]
    node_demand = train_d[:, 1:, :].squeeze()
    
    if save_path:
        data = {
            'depot_xy': depot_xy,
            'node_xy': node_xy,
            'node_demand': node_demand,
            'adj_matrices': adj_matrices,
            'seed': seed
        }
        torch.save(data, save_path)
        print(f"save into {save_path}，seed: {seed}")
    
    return depot_xy, node_xy, node_demand, adj_matrices

def augment_xy_data_by_8_fold(xy_data):
    # xy_data.shape: (batch, N, 2)

    x = xy_data[:, :, [0]]
    y = xy_data[:, :, [1]]
    # x,y shape: (batch, N, 1)

    dat1 = torch.cat((x, y), dim=2)
    dat2 = torch.cat((1 - x, y), dim=2)
    dat3 = torch.cat((x, 1 - y), dim=2)
    dat4 = torch.cat((1 - x, 1 - y), dim=2)
    dat5 = torch.cat((y, x), dim=2)
    dat6 = torch.cat((1 - y, x), dim=2)
    dat7 = torch.cat((y, 1 - x), dim=2)
    dat8 = torch.cat((1 - y, 1 - x), dim=2)

    aug_xy_data = torch.cat((dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8), dim=0)
    # shape: (8*batch, N, 2)

    return aug_xy_data

class GridNetworkGenerator:
    def __init__(self, node_count: int, link_count: int, batch_seed):

        self.node_count = node_count  
        self.link_count = link_count
        self.grid_size = int(np.ceil(np.sqrt(node_count - 1)))  
        self.seed = batch_seed  
    
    def generate_initial_grid(self) -> tuple:

        rand = random.Random(self.seed)
        
        nodes = {}
        edges = []
        offset_range = 1 / (2 * (self.grid_size - 1))
        
        depot_x = rand.uniform(0, 1)
        depot_y = rand.uniform(0, 1)
        nodes[0] = (depot_x, depot_y)
        
        node_id = 1
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if node_id < self.node_count:
                    x = j / (self.grid_size - 1)
                    y = i / (self.grid_size - 1)
                    
                    x += rand.uniform(-offset_range, offset_range)
                    y += rand.uniform(-offset_range, offset_range)
                    
                    x = max(0, min(1, x))
                    y = max(0, min(1, y))
                    
                    nodes[node_id] = (x, y)
                    node_id += 1

        for i in range(self.grid_size):
            for j in range(self.grid_size):
                current_id = i * self.grid_size + j + 1
                if current_id >= self.node_count:
                    break
                if j < self.grid_size - 1:
                    right_id = i * self.grid_size + (j + 1) + 1
                    if right_id < self.node_count:
                        edges.append((current_id, right_id))
                if i < self.grid_size - 1:
                    down_id = (i + 1) * self.grid_size + j + 1
                    if down_id < self.node_count:
                        edges.append((current_id, down_id))
        
        return nodes, edges
        
    def prune_edges(self, edges: list, nodes: int) -> list:

        rand = random.Random(self.seed)
        
        G = nx.Graph(edges)
        if not nx.is_connected(G):
            raise ValueError("Error, graph is not connected")
        
        actual_node_count = nodes - 1  
        min_edges = actual_node_count - 1 

        center_range = max(1, int(nodes * 0.25))  
        center_nodes = list(range(center_range, nodes - center_range))
        
        if center_nodes:
            start_node = rand.choice(center_nodes)
        else:
            start_node = rand.choice(list(G.nodes()))
        
        visited = {start_node}
        tree_edges = []
        queue = [start_node]
        
        while queue and len(tree_edges) < min_edges:
            node = queue.pop(0)
            neighbors = list(G.neighbors(node))
            rand.shuffle(neighbors)
            for neighbor in neighbors:
                if neighbor not in visited:
                    tree_edges.append((node, neighbor))
                    visited.add(neighbor)
                    queue.append(neighbor)
                    if len(tree_edges) == min_edges:
                        break
        
        need_add = self.link_count - min_edges
        
        all_edges = set(edges)
        tree_set = set([(u, v) for u, v in tree_edges]) | set([(v, u) for u, v in tree_edges])
        non_tree_edges = [e for e in all_edges if e not in tree_set]
        
        return tree_edges + rand.sample(non_tree_edges, need_add)
        
    def split_edges(self, nodes: dict, edges: list) -> tuple:
        
        rand = random.Random(self.seed)
        
        all_nodes = nodes.copy()
        new_edges = []
        next_id = max(nodes.keys()) + 1
        
        for a, b in edges:
            xa, ya = nodes[a]
            xb, yb = nodes[b]
            xc, yc = (xa+xb)/2, (ya+yb)/2
            
            length = np.sqrt((xb-xa)** 2 + (yb-ya)** 2)
            if length > 0:
                perp_x = -(yb-ya)/length
                perp_y = (xb-xa)/length
                dev = rand.uniform(-length*0.3, length*0.3)
                xc += perp_x * dev
                yc += perp_y * dev
            
            xc_reflected, yc_reflected = xc, yc
            
            if xc < 0:
                xc_reflected = -xc  
            elif xc > 1:
                xc_reflected = 2 - xc  
            
            if yc < 0:
                yc_reflected = -yc  
            elif yc > 1:
                yc_reflected = 2 - yc
            
            xc = max(0, min(1, xc_reflected))
            yc = max(0, min(1, yc_reflected))
            
            all_nodes[next_id] = (xc, yc)
            new_edges.append((a, next_id))
            new_edges.append((next_id, b))
            next_id += 1
        
        return all_nodes, new_edges

    def create_adjacency_matrix(self, nodes: dict, edges: list, original_ids: list) -> np.ndarray:
        
        n = len(nodes)
        adj = np.zeros((n, n), dtype=int)
        node2idx = {node_id: i for i, node_id in enumerate(sorted(nodes.keys()))}
        
        for u, v in edges:
            i, j = node2idx[u], node2idx[v]
            adj[i, j] = 1
            adj[j, i] = 1

        for idx in original_ids:
            for idy in original_ids:
                if idx != idy:  
                    adj[idx, idy] = 1
                    adj[idy, idx] = 1
        
        return adj

    def generate_demands(self, original_ids: list, new_ids: list) -> dict:

        rand = random.Random(self.seed)
        
        demands = {oid: 0 for oid in original_ids}
        demands.update({nid: rand.uniform(1, 10)/10 for nid in new_ids})
        return demands

    def generate_single_network(self) -> tuple:

        original_nodes, initial_edges = self.generate_initial_grid()
        pruned_edges = self.prune_edges(initial_edges, self.node_count)
        all_nodes, final_edges = self.split_edges(original_nodes, pruned_edges)
        
        original_ids = list(original_nodes.keys())
        new_ids = set(all_nodes.keys()) - set(original_ids)
        adj_matrix = self.create_adjacency_matrix(all_nodes, final_edges, original_ids)
        demands = self.generate_demands(original_ids, new_ids)
        
        sorted_ids = sorted(all_nodes.keys())
        coords = np.array([all_nodes[oid] for oid in sorted_ids], dtype=np.float32)
        node_demands = np.array([demands[oid] for oid in sorted_ids], dtype=np.float32).reshape(-1, 1)
        
        return coords, node_demands, adj_matrix

def generate_network_batch(batch_size, problem_size, original_node_count, link_count, seed=1234):

    s = torch.zeros(batch_size, problem_size+1, 2, dtype=torch.float32)
    d = torch.zeros(batch_size, problem_size+1, 1, dtype=torch.float32)
    adj_matrices = torch.zeros(batch_size, problem_size+1, problem_size+1, dtype=torch.float32)
    
    for i in range(batch_size):

        batch_seed = seed + i
        generator = GridNetworkGenerator(original_node_count, link_count, batch_seed)
        coords, demands, adj = generator.generate_single_network()
        s[i] = torch.from_numpy(coords)
        d[i] = torch.from_numpy(demands)
        adj_matrices[i] = torch.from_numpy(adj)
    
    return s, d, adj_matrices

def load_data(file_path):

    data = torch.load(file_path)
    print(f"load from {file_path}，seed: {data['seed']}")
    return (data['depot_xy'], data['node_xy'], data['node_demand'], data['adj_matrices'])

if __name__ == "__main__":
    # 生成并保存数据
    batch_size = 128
    problem_size = 797
    original_node_count = 398
    link_count = 400
    seed = 12345
    save_dir = r"C:\Users\gongh\POMO+road_network+GPU\POMO-master\NEW_py_ver\CVRP"
    file_name = f"network_data_{batch_size}_{problem_size+1}_{seed}.pt"
    save_path = os.path.join(save_dir, file_name)
    
    depot_xy, node_xy, node_demand, adj_matrices = get_random_problems(
        batch_size, problem_size, original_node_count, link_count, 
        seed=seed, save_path=save_path
    )
    


数据已保存到 C:\Users\gongh\POMO+road_network+GPU\POMO-master\NEW_py_ver\CVRP\network_data_128_798_12345.pt，使用的基础种子为 12345


In [10]:
loaded_depot_xy, loaded_node_xy, loaded_node_demand, loaded_adj_matrices = load_data(save_path)

数据已从 C:\Users\gongh\POMO+road_network+GPU\POMO-master\NEW_py_ver\CVRP\network_data_128_798_12345.pt 加载，使用的基础种子为 12345


In [11]:
depot_xy = loaded_depot_xy[0][None,:,:].repeat(128,1,1)
node_xy = loaded_node_xy[0][None,:,:].repeat(128,1,1)
node_demand = loaded_node_demand[0][None,:,].repeat(128,1)
adj_matrices = loaded_adj_matrices[0][None,:,:].repeat(128,1,1)

In [12]:
# 保存数据到文件
if save_path:
    data = {
        'depot_xy': depot_xy,
        'node_xy': node_xy,
        'node_demand': node_demand,
        'adj_matrices': adj_matrices,
        'seed': seed
    }
    torch.save(data, save_path)
    print(f"数据已保存到 {save_path}，使用的基础种子为 {seed}")

数据已保存到 C:\Users\gongh\POMO+road_network+GPU\POMO-master\NEW_py_ver\CVRP\network_data_128_798_12345.pt，使用的基础种子为 12345


In [13]:
loaded_adj_matrices.shape

torch.Size([128, 798, 798])

In [25]:
loaded_node_demand = loaded_node_demand[0][None,:,].repeat(128,1)

In [7]:
loaded_node_demand.shape

torch.Size([128, 97])

In [8]:
loaded_node_demand[0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.9698, 0.4967, 0.1067, 0.9199, 0.9453, 0.6240, 0.7044,
        0.1755, 0.7898, 0.3131, 0.1277, 0.8099, 0.4115, 0.6610, 0.6542, 0.2337,
        0.2648, 0.2030, 0.1132, 0.5381, 0.9684, 0.1581, 0.5870, 0.5193, 0.6413,
        0.1800, 0.6211, 0.3426, 0.6008, 0.6802, 0.5329, 0.4197, 0.3242, 0.9402,
        0.5080, 0.5771, 0.1174, 0.5573, 0.1052, 0.2294, 0.5255, 0.4396, 0.1488,
        0.6288, 0.2476, 0.6016, 0.2298, 0.9436, 0.7939, 0.9612])

In [9]:
loaded_node_demand[0][47]

tensor(0.9698)

In [10]:
a = {}
for i in range(0,len(loaded_node_demand[0])):
    a[i] = loaded_node_demand[0][i]
a

{0: tensor(0.),
 1: tensor(0.),
 2: tensor(0.),
 3: tensor(0.),
 4: tensor(0.),
 5: tensor(0.),
 6: tensor(0.),
 7: tensor(0.),
 8: tensor(0.),
 9: tensor(0.),
 10: tensor(0.),
 11: tensor(0.),
 12: tensor(0.),
 13: tensor(0.),
 14: tensor(0.),
 15: tensor(0.),
 16: tensor(0.),
 17: tensor(0.),
 18: tensor(0.),
 19: tensor(0.),
 20: tensor(0.),
 21: tensor(0.),
 22: tensor(0.),
 23: tensor(0.),
 24: tensor(0.),
 25: tensor(0.),
 26: tensor(0.),
 27: tensor(0.),
 28: tensor(0.),
 29: tensor(0.),
 30: tensor(0.),
 31: tensor(0.),
 32: tensor(0.),
 33: tensor(0.),
 34: tensor(0.),
 35: tensor(0.),
 36: tensor(0.),
 37: tensor(0.),
 38: tensor(0.),
 39: tensor(0.),
 40: tensor(0.),
 41: tensor(0.),
 42: tensor(0.),
 43: tensor(0.),
 44: tensor(0.),
 45: tensor(0.),
 46: tensor(0.),
 47: tensor(0.9698),
 48: tensor(0.4967),
 49: tensor(0.1067),
 50: tensor(0.9199),
 51: tensor(0.9453),
 52: tensor(0.6240),
 53: tensor(0.7044),
 54: tensor(0.1755),
 55: tensor(0.7898),
 56: tensor(0.3131),
 

In [11]:
loaded_adj_matrices.shape

torch.Size([128, 98, 98])